In [5]:
import os
import multiprocessing
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

part_name = "Living_hinge_1.step"
meshing_accuracy = 3
displacement_magnitude =  -1.000e-2

# Modify the geo file
with open("Living_hinge_1.geo", "r") as file:
    lines = file.readlines()

with open("Living_hinge_1.geo", "w") as file:
    for line in lines:
        if line.strip().startswith("Merge"):
            file.write(f'Merge "{part_name}";\n')
        elif line.strip().startswith("Mesh.CharacteristicLengthMax"):
            file.write(f"Mesh.CharacteristicLengthMax = {meshing_accuracy};\n")
        else:
            file.write(line) 

# Enable multithreading for ccx
os.environ['OMP_NUM_THREADS'] = str(multiprocessing.cpu_count())
snap = os.listdir(os.curdir)
os.system("cgx -b Living_hinge_1.fbd")
    
# Read mesh file and group nodes
def group_faces(node_coords):
    x = [coord[0] for coord in node_coords.values()]
    group1 = {}
    group2 = {}
    group1_threshold = 40
    group2_threshold = -40
    tolerance = 0.00005

    for key, values in node_coords.items():
        if np.abs(values[0] - group1_threshold) < tolerance:
            group1[key] = values
        elif np.abs(values[0] - group2_threshold) < tolerance:
            group2[key] = values

    return {"group1": group1, "group2": group2}

node_coords = {}
elements = []
plot_faces = False

with open("all.msh", "r") as file:
    in_node_section = False
    in_element_section = False 
    for line in file:
        if line.startswith("*NODE"):
            in_node_section = True
            in_element_section = False 
            continue
        if line.startswith("*ELEMENT"):
            in_node_section = False 
            in_element_section = True
            continue
        if in_node_section:
            parts = line.strip().split(",")
            if len(parts) >= 4:
                node_id = int(parts[0])
                x, y, z = map(float, parts[1:4])
                node_coords[node_id] = (x, y, z)
        if in_element_section and not line.startswith("*"):
            parts = [int(p) for p in line.split(",")]
            element_id = parts[0]
            node_ids = parts[1:]
            if len(node_ids) == 10:
                elements.append(node_ids)

grouped_faces = group_faces(node_coords)
upper_nset = list(grouped_faces["group1"].keys())
lower_nset = list(grouped_faces["group2"].keys())

# Save node sets
with open("upper.nam", "w") as file:
    file.write('**Nodes corresponding to the top surface\n*NSET, NSET=Nupper\n')
    for node in upper_nset:
        file.write(f'    {node}\n')

with open("lower.nam", "w") as file:
    file.write('**Node corresponding to the bottom surface\n*NSET, NSET=Nlower\n')
    for node in lower_nset:
        file.write(f'    {node}\n')


with open("disp.inc", "w") as file:
    file.write(f'Nupper, 1, 1, {displacement_magnitude:.4e}\n')

os.system("cgx -b run.fbd")

# Separate coordinates by label sets

on a Darwin machine, nodename Mac.attlocal.net, release 24.5.0, version Darwin Kernel Version 24.5.0: Tue Apr 22 19:54:26 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T8112, machine arm64 
 The HOME was detected:/Users/hugopenichou
parameters:3 arguments:2
GL_MAX_EVAL_ORDER:30, mesh threads:1
 /Users/hugopenichou/.cgx opened

 reading file
 done 

 Living_hinge_1.fbd opened

 reading file
 # sys gmsh part.step -clcurv -3 -o gmsh.inp -order 2
******************************************************************************
*  read ccx input                                   4.4.2002 wi              *
*                                                                            *
*                                                                            *
* -only free-format is supported. fixed format might cause a prg-crash.      *
*                                                                            *
*                                                                            *

on a Darwin machine, nodename Mac.attlocal.net, release 24.5.0, version Darwin Kernel Version 24.5.0: Tue Apr 22 19:54:26 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T8112, machine arm64 
 The HOME was detected:/Users/hugopenichou
parameters:3 arguments:2
GL_MAX_EVAL_ORDER:30, mesh threads:1
 /Users/hugopenichou/.cgx opened

 reading file
 done 

 run.fbd opened

 reading file
 # solve
 # read results
 file:Living_hinge_1.frd opened
 reading frd format
 MODEL NAME:  record:    1UUSER                                                              
record:    1UDATE              12.july.2025                                    
record:    1UTIME              23:12:59                                        
record:    1UHOST                                                              
record:    1UPGM               CalculiX                                        
record:    1UVERSION           Version 2.22                             
record:    1UCOMPILETIME       Mon Aug  5 19:15:25 CE

0